In [1]:
import os
os.environ["SPARK_HOME"] = "/Users/zouhairhajji/Documents/dev/spark-2.4.0-bin-hadoop2.7" 

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, DoubleType
import operator

import re

spark = SparkSession.builder \
        .master('local[*]') \
        .appName('Matrix multiplication') \
        .config("spark.driver.memory", "2g") \
        .enableHiveSupport() \
        .getOrCreate()


# Spark algorithm

In [6]:
file_regex = '../inputs/*.demo'
flat_data = spark.sparkContext.wholeTextFiles(file_regex)
flat_data.take(1)

[('file:/Users/zouhairhajji/Documents/git_projects/Data_science/words_co-occurante_matrix/inputs/file1.demo',
  "je suis Zouhair HAJJI et j'ai 4 ans\net j'ai mal aux dents\net j'ai du mal à faire le ramadan,\nmais je ne peux pas faire le ramadan parce que j'ai trop mal au gents dents\n")]

In [5]:
files = flat_data.map(lambda x: x[0].split('/')[-1]).collect()

In [10]:
def filter_appropriate_words(x):
    splitet_str = re.split('\n| |\'', x)
    
    splitet_str = list(filter(None, splitet_str))
    splitet_str = list(filter(lambda x: False if len(x) < 3 else True, splitet_str))
    
    return splitet_str

formated_input = flat_data    \
    .map(lambda x: (x[0].split('/')[-1],  filter_appropriate_words(x[1])    ))    \
    .flatMap(lambda x: [ (x[0], word) for word in x[1]]  )


result = formated_input     \
    .cartesian(formated_input)   \
    .filter(lambda x: x[0][0] == x[1][0])  \
    .filter(lambda x: x[0][1] != x[1][1])  \
    .map(lambda x:  ( (x[0][0], x[0][1], x[1][1]), 1)    ) \
    .reduceByKey(lambda x, y: x+y)   \
    .map(lambda x: (x[0][0], (x[0][1], x[0][2]), x[1]-1))  

In [14]:
result.saveAsTextFile('output_spark')

In [2]:
sc = spark.sparkContext

In [49]:
pairs = sc.parallelize([("a", 1), ("a", 5), ("b", 2), ("a", 8)]) 
pairs.reduceByKey(lambda x, y: y+1).collect()



def T(x):
    if x < 4:
        return [x]
    else :
        return []


dpairs = pairs.flatMap(lambda x: T(x[1])).map(lambda x: (x ,1))

print( pairs.collect() )
print( dpairs.collect() )

[('a', 1), ('a', 5), ('b', 2), ('a', 8)]
[(1, 1), (2, 1)]


In [3]:
names = ['A', 'B', 'C', 'D']
document =[['A', 'B'], ['C', 'B', 'K'],['A', 'B', 'C', 'D', 'Z']]

for a in names:
    for b in names:
        count = 0

        for x in document:
            if a != b:
                if a in x and b in x:
                    count += 1

            else:
                n = x.count(a)
                if n >= 2:
                    count += math.factorial(n)/math.factorial(n - 2)/2

        print ('{} x {} = {}'.format(a, b, count) )

A x A = 0
A x B = 2
A x C = 1
A x D = 1
B x A = 2
B x B = 0
B x C = 2
B x D = 1
C x A = 1
C x B = 2
C x C = 0
C x D = 1
D x A = 1
D x B = 1
D x C = 1
D x D = 0
